# Python startup 

In [ ]:
def reverse(value):
    output = value[::-1]
    return output

string = reverse("hello")
print(string)

### Read csv file and write it to another csv or txt

In [ ]:
import csv

with open(r'C:\Users\pavan\OneDrive\Desktop\DataEng\Python\california_housing_test.csv', mode = 'r') as file:
    data = csv.reader(file)
    
    with open('output.csv', mode = 'w') as output_file:
        output_writer=csv.writer(output_file,delimiter='-')
        for rec in data:
            output_writer.writerow(rec)

   
file.close()
output_file.close()


### Pandas and its funtions 

In [ ]:
import pandas as pd

df = pd.read_csv('output.csv')

# print("Raw data\n",df.to_string())

# new_df = df.dropna()
# print("altered data:\n",new_df.to_string())

# new_df = df.fillna('filled')
# print("new data\n", new_df)

#df['Age'].fillna(100, inplace=True)
# print("tranformed data\n",df)

avg_age = df['Age'].add_suffix('|')
print(avg_age.sort_values(ascending=True))

In [ ]:
def nth(data,n):
    
    df = pd.DataFrame(data,columns=['Id', 'Salary'])
    print("data:\n",df)
    print("length of data:",len(df))
    
    sorted_data = df['Salary'].sort_values(ascending=False)
    print(sorted_data)
    
    if len(sorted_data)<n:
        print("There is no {} highest salary".format(n))
    else:
        print('{} highest salary is:'.format(n),sorted_data.iloc[n])


data = [[12, 900], [2, 200], [3, 300],[9,300]]
result = nth(data,20)


In [ ]:
data = {'Department': ['Sales', 'Sales', 'HR', 'HR', 'Engineering', 'Engineering'],
 'Salary': [60000, 80000, 75000, 65000, 72000, 90000]}
df = pd.DataFrame(data)

df = df.groupby('Department')['Salary'].max().reset_index()

df = df[df['Salary']> 82000]

print(df)

In [ ]:
import numpy as np

dt = [1,2,3,4,5]

arr = np.zeros(5)
print(arr)

In [ ]:
import psycopg2
import pandas as pd
import csv

    
cursor = None 
connection = None 
country=[]

try:
    connection = psycopg2.connect(database="postgres", user="postgres", password="root", host='192.168.5.56', port=5432)
    cursor = connection.cursor()

    #sql section 
    sel_script = ''' select * from country; '''
    cursor.execute(sel_script)
    rows = cursor.fetchall()
    
    #sql to Data Frame
    df = pd.DataFrame(rows)
    df = df.sort_values(by=0,ascending=True)
    limit = df.__len__()    
    
    #output section 
    with open("country.txt",mode = 'w') as of:
        of_writer=csv.writer(of)
        for i in range(0,limit):
            x=df.iloc[i]
            x= x.str.upper()
            of_writer.writerow(x)
            
    print("{} file created".format("country.txt"))
    
    #New table definition
    table_name = 'country_new'
    column1 = 'id'
    column2 = 'name'
    
    try:
        sql_create = '''create table if not exists 
                            {} (
                            {} INT,
                            {} VARCHAR(20)
                                );
                        truncate table {};'''.format(table_name,column1,column2,table_name)
                                
        cursor.execute(sql_create)
        connection.commit() 

        print("Table created")
    except Exception as error:
            print("table created failed:",error)
        

    try:
        id=0  
        for rec in range (0,limit-1):
            id+=1
            val=df[0][rec]
            val=val.upper()
            sql_insert = ''' insert into {} ({},{})
                                    values (%s,%s);
                            '''.format(table_name,column1,column2)
            
            cursor.execute(sql_insert,(id,val))
            connection.commit()
        
        print("Records inserted:",id)    
    except Exception as error:
        print("table insertion failed:",error)
        

except Exception as error:
    print(error)
finally:
    if cursor and connection is not None:
        of.close()
        cursor.close()
        connection.close()
        print("Program execution complete")    

### python and snowflake

In [ ]:
import pandas as pd
import snowflake.connector
import csv


con = snowflake.connector.connect(
    account='RGXQNUE-AW70262',
    user='SAI',
    password='Plmnko@5',
    warehouse='COMPUTE_WH',
    database='DBT_SNOWFLAKE',
    schema='DBT_SNOWFLAKE_SCHEMA'
    )

cur = con.cursor()

sql_script = "select * from chargeback"

cur.execute(sql_script)
rows = cur.fetchall()
df = pd.DataFrame(rows)

print(df)



### Python and PDF

In [ ]:
import re
from pypdf import PdfReader
import pandas as pd


lst,dates,cards,case_number,Address,amount,folio_no = [[] for _ in range(7)]


folio = PdfReader(r'C:\Users\pavan\OneDrive\Desktop\DataEng\Python\Docs\Crystal Reports - rptFolio1.rpt.pdf')
cover = PdfReader(r'C:\Users\pavan\OneDrive\Desktop\DataEng\Python\Docs\Cover Letter.pdf')

# print('Folio letter total pages: ',len(folio.pages))
# print('Cover letter total pages: ',len(cover.pages))

cover_lines = cover.pages[0].extract_text()

cover_lines=cover_lines.splitlines()

#regex pattern
pattern_card = r'(\d{4})'
pattern_date = r'(\d{2}/\d{2}/\d{2})'
pattern_case = r'(CB\d)'
pattern_folio = "[0-9]"
pattern_amount = "[$]+[0-9]+[.]+[0-9]"

row = 0
for case in cover_lines:
    
    if case != ' ' and row <=2:
        Address.append(case)
        row+=1
    
    lst = case.split(' ')
    
    for i in lst:
        
        #fetch card
        match = re.search(pattern_card,i)
        if match:
            i= i.strip(',').strip(' ').strip(').')
            if len(i) == 4:
                cards.append(i) 
        
        
        #fetch case
        i=i.upper()
        match = re.search(pattern_case,i)
        if match:
            case_number.append(i)
            
        #fetch folio
        i=i.upper()
        match = re.search(pattern_folio,i)
        if match and i.isdigit() and len(i)>5:
            folio_no.append(i)
        
            
        #fetch date
        match = re.search(pattern_date,i)
        if match:
            dates.append(i)
            
        #fetch amount
        i=i.upper()
        match = re.search(pattern_amount,i)
        if match:
            amount.append(i)
        
print("case",case_number)
print("Dates", dates)
print("card", cards )
print("Address",Address)
print("Amount",amount)
print("folio no",folio_no)
        
        

### Folio

In [ ]:
import re
from pypdf import PdfReader
import pandas as pd
import pdfplumber

folio = PdfReader(r"C:\Users\pavan\OneDrive\Desktop\DataEng\Python\Docs\Crystal Reports - rptFolio1.rpt.pdf")

raw_text = folio.pages[0].extract_text()
lines = raw_text.splitlines()
pattern = "[a-z-A-Z0-9]+@[a-zA-Z]"

name = ['Name']
folio = ['Folio']

for line in lines:
    words = line.split(' ')  
    print(words)
    for word in words:
        email = re.search(pattern,word)
        if email:
            print(word)
            email = word
        elif word in ['Date','Out','In']:
            if word == 'Out':
                date_out = words[2]
                print(date_out)
            elif word == 'In':
                date_in = words[2]
                print(date_in)
        
        else:
            continue
        
        
        

In [ ]:

'''

temp=[]
headers = []
row1 = []
row2 = []
str=''
with pdfplumber.open(r"C:\Users\pavan\OneDrive\Desktop\DataEng\Python\Docs\Crystal Reports - rptFolio1.rpt.pdf") as f: 
    tables = f.pages[0].extract_table()
    text  = f.pages[0].extract_text()
    for i in range(len(tables)):
        for var in tables[i]:
            if i == 0:
                if var!=None:
                    headers.append(var)
            elif i==1:
                temp = var.split('\n')
                row1.append(temp)
            else:
                temp = var.split('\n')
                row2.append(temp)


column_names = ['Case_No','Name', 'Address','Phone', 'Country', 'ID_Type','Folio_No','DateIn','Dateout','Transaction_Date','Amount']
df = pd.DataFrame(headers).transpose()
df1 = pd.DataFrame(row1).transpose()
df2 = pd.DataFrame(row2).transpose()

df.columns = column_names

df
'''